In [2]:
import shutil
import subprocess
import os

os.chdir("..")

In [4]:
def get_len_dat(path):
    with open(path, "r") as file:
        return len(file.readlines())

In [6]:
POSITIVES_DAT = "dataset/licenseplates/positive_0.dat"
NEGATIVES_DAT = "dataset/licenseplates/negative.dat"

WORK_DIR = "models"
VEC_FILE = os.path.join(WORK_DIR, "positives.vec")

CASCADE_DIR = os.path.join(WORK_DIR, "cascade")

POSITIVE_DAT_LEN = get_len_dat(POSITIVES_DAT)
NEGATIVE_DAT_LEN = get_len_dat(NEGATIVES_DAT)

NUM_NEG = 2500
NUM_STAGES = 10
MIN_HIT_RATE = 0.995
NUM_POS = POSITIVE_DAT_LEN * MIN_HIT_RATE**(NUM_STAGES - 1)

SAMPLE_WIDTH = 50
SAMPLE_HEIGHT = 50

OPENCV_CREATESAMPLES = "opencv/build/x64/vc15/bin/opencv_createsamples.exe"
OPENCV_TRAINCASCADE = "opencv/build/x64/vc15/bin/opencv_traincascade.exe"

if not os.path.exists(OPENCV_CREATESAMPLES):
    raise FileNotFoundError("opencv_createsamples executable not found.")
if not os.path.exists(OPENCV_TRAINCASCADE):
    raise FileNotFoundError("opencv_traincascade executable not found.")

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(CASCADE_DIR, exist_ok=True)

In [4]:
def execute_cmd(command) -> None:
    print("Executing:", " ".join(command))
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(result.stdout)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {result.returncode}")

In [5]:
if os.path.exists(VEC_FILE):
    os.remove(VEC_FILE)

command = [
    OPENCV_CREATESAMPLES,
    "-info", str(POSITIVES_DAT),
    "-vec", str(VEC_FILE),
    "-num", str(POSITIVE_DAT_LEN),
    "-w", str(SAMPLE_WIDTH),
    "-h", str(SAMPLE_HEIGHT),
]

execute_cmd(command)
print(f"Created vec file at {VEC_FILE}")

Executing: opencv/build/x64/vc15/bin/opencv_createsamples.exe -info dataset/licenseplates/positive_0.dat -vec models\positives.vec -num 1230 -w 50 -h 50
Info file name: dataset/licenseplates/positive_0.dat
Img file name: (NULL)
Vec file name: models\positives.vec
BG  file name: (NULL)
Num: 1230
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 50
Height: 50
Max Scale: -1
RNG Seed: 12345
Create training samples from images collection...
Done. Created 1230 samples

Created vec file at models\positives.vec


In [ ]:
shutil.rmtree(CASCADE_DIR, ignore_errors=True)

command = [
    OPENCV_TRAINCASCADE,
    "-data", str(CASCADE_DIR),
    "-vec", str(VEC_FILE),
    "-bg", str(NEGATIVES_DAT),
    "-numPos", str(NUM_POS),
    "-numNeg", str(NUM_NEG),
    "-numStages", str(NUM_STAGES),
    "-featureType", "HAAR",
    "-w", str(SAMPLE_WIDTH),
    "-h", str(SAMPLE_HEIGHT),
    "-mode", "ALL",
    "-minHitRate", "0.995",
    "-maxFalseAlarmRate", "0.5",
    "-precalcValBufSize", "1024",
    "-precalcIdxBufSize", "1024",
    "-numThreads", str(os.cpu_count() or 4),
]

execute_cmd(command)
print(f"Cascade training artifacts saved in {CASCADE_DIR}")

Executing: opencv/build/x64/vc15/bin/opencv_traincascade.exe -data models\cascade -vec models\positives.vec -bg dataset/licenseplates/negative.dat -numPos 1175.7441813797984 -numNeg 2500 -numStages 10 -featureType HAAR -w 50 -h 50 -mode ALL -minHitRate 0.995 -maxFalseAlarmRate 0.5 -precalcValBufSize 1024 -precalcIdxBufSize 1024 -numThreads 12
